<a href="https://colab.research.google.com/github/wandb/examples/blob/colabs%2Fraytune-torch/colabs/raytune/W%26B%20%2B%20RayTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq filelock==3.0.12
!pip install -U -qq ray==0.8.7
!pip install -qq wandb

In [ ]:
import getpass

import random
import numpy as np
import torch
import torch.optim as optim
from ray import tune
from ray.tune.examples.mnist_pytorch import get_data_loaders, ConvNet, train, test
from ray import tune
from ray.tune.integration.wandb import WandbLogger
from ray.tune.integration.wandb import wandb_mixin
import wandb

random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)

**W&B** integrates with `Ray` by offering two lightweight integrations:


1.   One is the `WandbLogger`, which automatically logs metrics reported to Tune to the Wandb API

2.  The other one is the `@wandb_mixin` decorator, which can be used with the function API. It automatically initializes the Wandb API with Tune’s training information.

You can just use the Wandb API like you would normally do, e.g. using `wandb.log()` to log your training process.

In [ ]:
@wandb_mixin
def train_mnist(config):
    '''
    Train_mnist function uses @wandb_mixin, so wandb.log() can be called within the scope of this function to report metrics.
    '''
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loader, test_loader = get_data_loaders()
    model = ConvNet()
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=config["lr"],momentum=config["momentum"])
    
    for i in range(10):
        train(model, optimizer, train_loader,device=device)
        acc = test(model, test_loader,device=device)

        metric = acc/2 # custom metric example
        #Utilizing @wandb_mixin
        wandb.log({"custom metric":metric})

        #When using WanddLogger, the metrics reported to tune are also logged in the W&B dashboard
        tune.report(mean_accuracy=acc)



In [ ]:
api_key = getpass.getpass("Enter your W&B apikey from https://wandb.ai/settings : ")
analysis = tune.run(
    train_mnist,
     
     loggers=[WandbLogger], # WandbLogger

     resources_per_trial={'gpu': 1},
     config={
        "monitor": True,
        #wandb dict accepts all arguments that can be passed in wandb.init() 
        "wandb": {"project": "rayTune-test", "monitor_gym": True,
                  "job_type": "GPU",
                  "api_key": api_key},
        "lr": tune.grid_search([0.0001,0.001,0.1,0.1,0.05,0.005]),
        "momentum": tune.grid_search([0.8,0.85,0.99, 0.9])       
    })

print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))

# Get a dataframe for analyzing trial results.
df = analysis.dataframe()